In [1]:
import h5py
import numpy as np
import csv
import eagle_python as eagle
import read_eagle as read_particle

In [23]:
r200=np.zeros([1])
m200=np.zeros([1])
pos=np.zeros([1,3])
Vel=np.zeros([1,3])
starmass=np.zeros([1])
FirstSubID=np.zeros([1])
for i in np.arange(256):
    f = h5py.File('/Simulations/Eagle/RefL0100N1504/groups_028_z000p000/eagle_subfind_tab_028_z000p000.'+np.str(i)+'.hdf5', 'r')
    r200=np.concatenate((r200,f['FOF']['Group_R_Crit200']))
    pos=np.concatenate((pos,f['FOF']['GroupCentreOfPotential']))
    m200=np.concatenate((m200,f['FOF']['Group_M_Crit200']))
    Vel=np.concatenate((Vel,f["Subhalo"]["Velocity"])) 
    starmass=np.concatenate((starmass,f['Subhalo']['Stars']['Mass']))     
    FirstSubID = np.hstack((FirstSubID,np.array(f["FOF"]["FirstSubhaloID"])))  
r200 = np.delete(r200, 0, 0)
Vel = np.delete(Vel, 0, 0)
pos = np.delete(pos,0,0)
m200 = np.delete(m200,0,0)
starmass=np.delete(starmass,0,0)
FirstSubID=np.delete(FirstSubID,0,0)

In [ ]:
basePath= '/Simulations/Eagle/RefL0100N1504/'
snapnum=28

fields=['Coordinates','Velocity']

for i in np.arange(len(FirstSubID)):
    SubID = FirstSubID[i]
    SubVel = Vel[int[SubID]]
    

In [18]:
len(FirstSubID)

1885062

In [26]:
FirstSubID[m200>1e-2]

array([      0.,   17982.,   37654., ..., 2275507., 2275508., 2275509.])

In [6]:
halo=np.load("/home/heyi/galaxy_spin_code/HeYi/spin_mass/revised_version/halo_pos_J_mass.npy")

In [4]:
basePath= '/Simulations/Eagle/RefL0100N1504/'
snapnum=28
fname   = eagle.snapshot.snapPath(basePath,snapnum,chunkNum=2)
print(fname)

/Simulations/Eagle/RefL0100N1504//snapshot_028_z000p000/snap_028_z000p000.2.hdf5


In [2]:
import numpy as np
import h5py
import eagle_python as eagle
import read_eagle as read_particle
from multiprocessing import Pool
import multiprocessing
from tqdm import trange, tqdm
import time
time_start=time.time()

#read data
r200=np.zeros([1])
mass=np.zeros([1])
pos=np.zeros([1,3])
GN=np.zeros([1])
SGN=np.zeros([1])
Vel=np.zeros([1,3])
starmass=np.zeros([1])

for i in np.arange(256):
    f = h5py.File('/Simulations/Eagle/RefL0100N1504/groups_028_z000p000/eagle_subfind_tab_028_z000p000.'+np.str(i)+'.hdf5', 'r')
    r200=np.concatenate((r200,f['FOF']['Group_R_Crit200']))

    pos=np.concatenate((pos,f['FOF']['GroupCentreOfPotential']))
    mass=np.concatenate((mass,f['FOF']['Group_M_Crit200']))
    GroupNumber=f["Subhalo"]["GroupNumber"]
    GN=np.concatenate((GN,GroupNumber))
    SubGroupNumber=f["Subhalo"]["SubGroupNumber"]
    SGN=np.concatenate((SGN,SubGroupNumber))
    Vel=np.concatenate((Vel,f["Subhalo"]["Velocity"])) 
    starmass=np.concatenate((starmass,f['Subhalo']['Stars']['Mass']))    
    
    
r200 = np.delete(r200, 0, 0)
Vel = np.delete(Vel, 0, 0)
pos = np.delete(pos,0,0)
mass = np.delete(mass,0,0)
GN = np.delete(GN,0,0)
SGN = np.delete(SGN,0,0)
starmass=np.delete(starmass,0,0)



main=np.int32(np.where(SGN==0))
GN=GN[main]
Vel=Vel[main] 
starmass=starmass[main]


GN=np.int32(GN-1)

GN=np.int32(GN)
r200=r200[GN]
pos=pos[GN]
mass=mass[GN] 

num=(np.where(mass>=1e-1))
r200=r200[num]
pos=pos[num]
mass=mass[num]
Vel=Vel[num]
starmass=starmass[num]

np.save("starmass.npy",starmass)

print("finish 1")
print("len(mass)=%s\n"%(len(mass)))

finish 1
len(mass)=393684

